In [1]:
import keras
from keras import layers, Input, optimizers, models
from keras.applications.resnet50 import ResNet50, decode_predictions
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.resnet50 import preprocess_input
img_size = (224, 224)
BATCH_SIZE = 1
NUM_EPOCHS = 100
DATAPATCH = '../2019_endoscopy_image/'
classes = ['normal', 'abnormal']

Using TensorFlow backend.


# Generate data set

In [2]:
train_datagen = ImageDataGenerator(preprocess_input,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,                     
         fill_mode='nearest')

train_batches = train_datagen.flow_from_directory(
    directory=DATAPATCH+'train', target_size=img_size, batch_size=BATCH_SIZE, class_mode='binary', classes=classes)
test_datagen = ImageDataGenerator(preprocess_input)
valid_batches = test_datagen.flow_from_directory(
    directory=DATAPATCH+'test', target_size=img_size, batch_size=BATCH_SIZE, class_mode='binary', classes=classes)
input_tensor = Input(shape=(124, 124, 3), dtype='float32', name='input')

W0910 17:54:33.016933 139769115248448 deprecation_wrapper.py:119] From /home/leeon/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



Found 2561 images belonging to 2 classes.
Found 450 images belonging to 2 classes.


# Load ResNet 50

In [3]:
from keras.applications.resnet50 import ResNet50
base_model = ResNet50(weights='imagenet', include_top=False,input_shape = (224, 224, 3))
print(base_model.summary())

W0910 17:54:33.068253 139769115248448 deprecation_wrapper.py:119] From /home/leeon/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0910 17:54:33.086916 139769115248448 deprecation_wrapper.py:119] From /home/leeon/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4185: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W0910 17:54:33.118762 139769115248448 deprecation_wrapper.py:119] From /home/leeon/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0910 17:54:33.119467 139769115248448 deprecation_wrapper.py:119] From /home/leeon/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

# TensorBoard

In [4]:
# Specify the Log Directory
import datetime as dt

# seriously? STRingFormatTIME? You gottabe kidding me
logdir = "logs/"+dt.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

# Define Classfier

In [5]:
from keras.layers import Dense, GlobalAveragePooling2D

from keras.models import Model
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(2048, activation='relu')(x)
# and a logistic layer --  we have 5 classes
predictions = Dense(1, activation='sigmoid')(x)

# this is the model we will train
model1 = Model(inputs=base_model.input, outputs=predictions)
model1.compile(optimizer=optimizers.SGD(lr=0.0001, momentum=0.9),
                 loss='binary_crossentropy', metrics=['accuracy'])

W0910 17:54:43.349946 139769115248448 deprecation_wrapper.py:119] From /home/leeon/.local/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0910 17:54:43.362040 139769115248448 deprecation.py:323] From /home/leeon/.local/lib/python3.6/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


# reduce Learning rate when stuff

In [6]:
from keras.callbacks import ReduceLROnPlateau


reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                   patience=8, min_lr=0.0000001, verbose=1,cooldown=1)


# fit model

In [ ]:

H = model1.fit_generator(train_batches,
                            steps_per_epoch = train_batches.samples // BATCH_SIZE,
                            validation_data = valid_batches,
                            validation_steps=valid_batches.samples // BATCH_SIZE,
                            epochs = NUM_EPOCHS,
                            callbacks=[tensorboard_callback,reduce_lr]).history


W0910 17:54:47.731171 139769115248448 deprecation_wrapper.py:119] From /home/leeon/.local/lib/python3.6/site-packages/keras/callbacks.py:850: The name tf.summary.merge_all is deprecated. Please use tf.compat.v1.summary.merge_all instead.

W0910 17:54:47.731788 139769115248448 deprecation_wrapper.py:119] From /home/leeon/.local/lib/python3.6/site-packages/keras/callbacks.py:853: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.



Epoch 1/100


/home/leeon/.local/lib/python3.6/site-packages/keras_preprocessing/image/image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


2561/2561 [==============================] - 187s 73ms/step - loss: 0.5916 - acc: 0.6865 - val_loss: 0.4368 - val_acc: 0.8311
Epoch 2/100
2561/2561 [==============================] - 175s 69ms/step - loss: 0.3956 - acc: 0.8282 - val_loss: 0.7812 - val_acc: 0.6422
Epoch 3/100
2561/2561 [==============================] - 176s 69ms/step - loss: 0.2935 - acc: 0.8735 - val_loss: 0.5423 - val_acc: 0.8222
Epoch 4/100
2561/2561 [==============================] - 176s 69ms/step - loss: 0.2271 - acc: 0.9137 - val_loss: 0.4122 - val_acc: 0.8778
Epoch 5/100
2561/2561 [==============================] - 168s 65ms/step - loss: 0.2131 - acc: 0.9188 - val_loss: 0.5053 - val_acc: 0.8422
Epoch 6/100
2561/2561 [==============================] - 173s 68ms/step - loss: 0.1945 - acc: 0.9254 - val_loss: 0.4617 - val_acc: 0.8822
Epoch 7/100
2561/2561 [==============================] - 184s 72ms/step - loss: 0.1858 - acc: 0.9274 - val_loss: 0.4994 - val_acc: 0.8533
Epoch 8/100
2561/2561 [=======================

## print out resualt

In [ ]:
history = H.history
loss = history['loss']
val_loss = history['val_loss']
acc = history['acc']
val_acc = history['val_acc']
epochs = range(1, len(loss)+1)
epochs2 = range(1,len(val_loss)+1)
plt.plot(epochs, loss, 'r', label='train loss')
plt.plot(epochs, acc, 'g', label='train acc')
plt.plot(epochs2, val_acc, 'b', label='Val acc')
plt.plot(epochs2, val_loss, 'c', label='Val loss')
plt.xlabel('Epochs')
plt.ylabel('Loss/Acc')
plt.legend()
plt.show()

In [ ]:
plt.plot(epochs, loss, 'r', label='train loss')
plt.plot(epochs2, val_loss, 'c', label='Val loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()
plt.plot(epochs, acc, 'g', label='train acc')
plt.plot(epochs2, val_acc, 'b', label='Val acc')
plt.xlabel('Epochs')
plt.ylabel('Acc')
plt.legend()
plt.show()

## resualt

In [ ]:
test_batches = train_datagen.flow_from_directory(DATAPATCH + 'test',
                                                  target_size=img_size,
                                                  class_mode='binary',
                                                  batch_size=BATCH_SIZE,
                                                 classes= classes ,
                                                 shuffle=False)
result = model1.evaluate_generator(generator=test_batches,steps = test_batches.samples)
print(result)